In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import missingno as msno
import seaborn as sns
import scipy.stats as st
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import xgboost as xg
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import accuracy_score, f1_score
from eli5.sklearn import PermutationImportance
from eli5 import show_weights
from sklearn.tree import DecisionTreeClassifier
from bayes_opt import BayesianOptimization

In [2]:
#Загрузим данные из файла.
df = pd.read_excel('/home/rik/Рабочий стол/МИФИ/учеба/мо/curse/data.xlsx', index_col=0)
df.head()

,"IC50, mM","CC50, mM",SI,MaxAbsEStateIndex,MaxEStateIndex,MinAbsEStateIndex,MinEStateIndex,qed,SPS,MolWt,...,fr_sulfide,fr_sulfonamd,fr_sulfone,fr_term_acetylene,fr_tetrazole,fr_thiazole,fr_thiocyan,fr_thiophene,fr_unbrch_alkane,fr_urea
0,6.239374,175.482382,28.125000,5.094096,5.094096,0.387225,0.387225,0.417362,42.928571,384.652,...,0,0,0,0,0,0,0,0,3,0
1,0.771831,5.402819,7.000000,3.961417,3.961417,0.533868,0.533868,0.462473,45.214286,388.684,...,0,0,0,0,0,0,0,0,3,0
2,223.808778,161.142320,0.720000,2.627117,2.627117,0.543231,0.543231,0.260923,42.187500,446.808,...,0,0,0,0,0,0,0,0,3,0
3,1.705624,107.855654,63.235294,5.097360,5.097360,0.390603,0.390603,0.377846,41.862069,398.679,...,0,0,0,0,0,0,0,0,4,0
4,107.131532,139.270991,1.300000,5.150510,5.150510,0.270476,0.270476,0.429038,36.514286,466.713,...,0,0,0,0,0,0,0,0,0,0


In [3]:
print("train.shape = {} rows, {} cols".format(*df.shape))

train.shape = 1001 rows, 213 cols


# Предобработка данных.

In [4]:
def transform(df): 
    discrete_feature = [feature for feature in df.columns if len(df[feature].unique())<25]
    count_discrete_feature = {}
    for i in tqdm(discrete_feature):
        cnt = 0
        for j in df[i]:
            if j != 0:
                cnt += 1
        count_discrete_feature[f'{i}'] = cnt
    count_discrete_feature = pd.Series(count_discrete_feature)
    df = df.drop(columns=count_discrete_feature[count_discrete_feature == 0].index.to_list())
    df = df.dropna()
    feature_del = ['BertzCT', 'Chi0', 'Chi0v', 'Chi1', 'Chi1n', 'Chi1v', 'Chi2v', 'Chi3n', 'Chi3v', 'Chi4n', 'Chi4v',
     'ExactMolWt', 'FpDensityMorgan1', 'FpDensityMorgan3', 'HeavyAtomCount', 'HeavyAtomMolWt', 'Kappa1', 'Kappa2',
     'Kappa3', 'LabuteASA', 'MaxAbsEStateIndex', 'MaxAbsPartialCharge', 'MolMR', 'MolWt', 'NHOHCount', 'NumAromaticCarbocycles',
     'NumHAcceptors', 'NumHeteroatoms', 'NumSaturatedCarbocycles', 'NumValenceElectrons', 'SMR_VSA4', 'SMR_VSA9',
     'SlogP_VSA6', 'TPSA', 'VSA_EState2', 'VSA_EState3', 'VSA_EState6', 'fr_Al_OH_noTert', 'fr_COO', 'fr_COO2',
     'fr_C_O_noCOO', 'fr_Nhpyrrole', 'fr_benzene', 'fr_nitro_arom_nonortho', 'fr_phenol', 'fr_phenol_noOrthoHbond',
    'MinAbsPartialCharge', 'Chi2n']
    df = df.drop(columns=feature_del)
   
    return df
    

In [5]:
df = transform(df)

100%|███████████████████████████████████████| 110/110 [00:00<00:00, 6375.73it/s]


In [6]:
val_median = df['CC50, mM'].median()
df['CC50, mM'] = df['CC50, mM'].apply(lambda x: 0 if x < val_median else 1)
df['CC50, mM'].head()

0    0
1    0
2    0
3    0
4    0
Name: CC50, mM, dtype: int64

## Обучение нескольких моделей, их сравнение. 

In [7]:
target = df['CC50, mM']
data = df.drop(['CC50, mM','IC50, mM','SI'], axis=1)

In [8]:
x_train, x_valid = train_test_split(
    data, train_size=0.8, random_state=1
)
y_train, y_valid = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid.shape))

x_train.shape = 798 rows, 144 cols
x_valid.shape = 200 rows, 144 cols


In [9]:
# Создадим модель дерева решений
tree = DecisionTreeClassifier(random_state=27)
# Создадим модель CatBoost
model = CatBoostClassifier(random_state=27)
# Создадим модель lightgbm
model_LGBM = LGBMClassifier(random_state=27)
# Создадим модель xgboost
xgb_r = xg.XGBClassifier() 
# обучение моделей
tree.fit(x_train, y_train)
model.fit(x_train, y_train, verbose=0)
model_LGBM.fit(x_train, y_train, verbose=0)
xgb_r.fit(x_train, y_train, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_tree_train = tree.predict(x_train)
y_pred_сat_Boost_train = model.predict(x_train)
y_pred_LGBM_train = model_LGBM.predict(x_train)
y_pred_xgb_train = xgb_r.predict(x_train)
# предсказание ответов для тестовой выборки
y_pred_tree = tree.predict(x_valid)
y_pred_сat_Boost = model.predict(x_valid)
y_pred_LGBM = model_LGBM.predict(x_valid)
y_pred_xgb = xgb_r.predict(x_valid)

print("Train tree f1_score: ", f1_score(y_train, y_pred_tree_train))
print("Train tree accuracy_score: ", accuracy_score(y_train, y_pred_tree_train))
print("Validation tree f1_score: ", f1_score(y_valid, y_pred_tree))
print("Validation tree accuracy_score: ", accuracy_score(y_valid, y_pred_tree))
print('*' * 50)
print("Train сat_Boost f1_score: ", f1_score(y_train, y_pred_сat_Boost_train))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train, y_pred_сat_Boost_train))
print("Validation сat_Boost f1_score: ", f1_score(y_valid, y_pred_сat_Boost))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid, y_pred_сat_Boost))
print('*' * 50)
print("Train LGBM f1_score: ", f1_score(y_train, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid, y_pred_LGBM))
print('*' * 50)
print("Train xgb f1_score: ", f1_score(y_train, y_pred_xgb_train))
print("Train xgb accuracy_score: ", accuracy_score(y_train, y_pred_xgb_train))
print("Validation xgb f1_score: ", f1_score(y_valid, y_pred_xgb))
print("Validation xgb accuracy_score: ", accuracy_score(y_valid, y_pred_xgb))

/usr/local/lib/python3.8/dist-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


Train tree f1_score:  0.9698492462311556
Train tree accuracy_score:  0.9699248120300752
Validation tree f1_score:  0.6907216494845362
Validation tree accuracy_score:  0.7
**************************************************
Train сat_Boost f1_score:  0.9486552567237163
Train сat_Boost accuracy_score:  0.9473684210526315
Validation сat_Boost f1_score:  0.7821782178217821
Validation сat_Boost accuracy_score:  0.78
**************************************************
Train LGBM f1_score:  0.9702970297029703
Train LGBM accuracy_score:  0.9699248120300752
Validation LGBM f1_score:  0.75
Validation LGBM accuracy_score:  0.75
**************************************************
Train xgb f1_score:  0.9702970297029703
Train xgb accuracy_score:  0.9699248120300752
Validation xgb f1_score:  0.75
Validation xgb accuracy_score:  0.75


Из приведенных данных выше мы видим хорошее качество моделей, а также переобучение. Наилучшие результаты показали модели сat_Boost, LGBM и XGBClassifier. С ними и продолжим работать.

# Подбор признаков, их анализ и оценка важности.

**CatBoost**

In [10]:
col = x_train.columns.tolist()
estimator_cat_boost = PermutationImportance(
    model, scoring= "f1", random_state=27
)
importance_cat_boost = estimator_cat_boost.fit(x_train, y_train)

In [11]:
show_weights(importance_cat_boost, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0063 ± 0.0071,EState_VSA4
0.0056 ± 0.0072,PEOE_VSA6
0.0048 ± 0.0045,fr_NH2
0.0045 ± 0.0052,MinPartialCharge
0.0044 ± 0.0051,EState_VSA3
0.0043 ± 0.0033,BalabanJ
0.0041 ± 0.0024,MinAbsEStateIndex
0.0040 ± 0.0028,AvgIpc
0.0035 ± 0.0048,BCUT2D_MWLOW
0.0032 ± 0.0011,PEOE_VSA10


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [12]:
feature_cat_boost = ['EState_VSA4','PEOE_VSA6','fr_NH2','MinPartialCharge','EState_VSA3','BalabanJ','MinAbsEStateIndex',
'AvgIpc','BCUT2D_MWLOW','PEOE_VSA10','EState_VSA2','qed','BCUT2D_MRLOW','SlogP_VSA1','SMR_VSA5','BCUT2D_LOGPLOW',
'MolLogP','NumSaturatedHeterocycles','VSA_EState7','VSA_EState5','NumRotatableBonds','fr_C_O','SMR_VSA6','BCUT2D_MRHI',
'EState_VSA5','PEOE_VSA1','BCUT2D_CHGLO','EState_VSA8','EState_VSA7','PEOE_VSA8','EState_VSA10','SlogP_VSA3',
'SlogP_VSA5','VSA_EState8','PEOE_VSA9','VSA_EState1','Chi0n','SMR_VSA10','PEOE_VSA7','fr_unbrch_alkane',
'fr_urea','fr_bicyclic','fr_ether','SlogP_VSA2','SMR_VSA7','fr_aryl_methyl','fr_ester','fr_ketone_Topliss',
'NumAliphaticRings','PEOE_VSA13','MinEStateIndex','MaxEStateIndex','Ipc','EState_VSA6','FractionCSP3','PEOE_VSA3',
'SlogP_VSA4','fr_allylic_oxid','NumAromaticHeterocycles','NumAliphaticCarbocycles','NumAliphaticHeterocycles',
'SlogP_VSA8','BCUT2D_MWHI','HallKierAlpha','SMR_VSA1','PEOE_VSA2','PEOE_VSA12','SMR_VSA3','VSA_EState10',
'fr_alkyl_halide','fr_NH0','VSA_EState4','SPS','fr_sulfide','BCUT2D_LOGPHI','PEOE_VSA11','SlogP_VSA12',
'VSA_EState9','EState_VSA9','BCUT2D_CHGHI','PEOE_VSA4','NOCount','fr_Imine','NumSaturatedRings','fr_morpholine',
'RingCount','fr_Ndealkylation1','fr_para_hydroxylation','fr_piperdine','FpDensityMorgan2','fr_ArN','MaxPartialCharge',
'fr_Ar_OH',]

In [13]:
target = df['CC50, mM']
data_cb = data[feature_cat_boost]

In [14]:
x_train_cb, x_valid_cb = train_test_split(
    data_cb, train_size=0.8, random_state=1
)
y_train_cb, y_valid_cb = train_test_split(
    target, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_cb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_cb.shape))

x_train.shape = 798 rows, 93 cols
x_valid.shape = 200 rows, 93 cols


In [15]:
# Создадим модель CatBoost
model = CatBoostClassifier(random_state=27)
# обучение моделей
model.fit(x_train_cb, y_train_cb, verbose=0)
# предсказание ответов для тренеровочной выборки
y_pred_сat_Boost_train = model.predict(x_train_cb)
# предсказание ответов для тестовой выборки
y_pred_сat_Boost = model.predict(x_valid_cb)


print("Train сat_Boost f1_score: ", f1_score(y_train_cb, y_pred_сat_Boost_train))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train_cb, y_pred_сat_Boost_train))
print("Validation сat_Boost f1_score: ", f1_score(y_valid_cb, y_pred_сat_Boost))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid_cb, y_pred_сat_Boost))


Train сat_Boost f1_score:  0.9509803921568627
Train сat_Boost accuracy_score:  0.949874686716792
Validation сat_Boost f1_score:  0.7821782178217821
Validation сat_Boost accuracy_score:  0.78


Отобрав признаки улучшить метрику не удалось, но удалось уменьшить размерность датасета со 144 признаков до 93.

**LGBM**

In [16]:
col = x_train.columns.tolist()
estimator_LGBM = PermutationImportance(
    model_LGBM, scoring= "f1", random_state=27
)
importance_LGBM = estimator_LGBM.fit(x_train, y_train)


In [17]:
show_weights(importance_LGBM, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0094 ± 0.0074,PEOE_VSA6
0.0061 ± 0.0050,MolLogP
0.0049 ± 0.0120,MinPartialCharge
0.0037 ± 0.0079,SMR_VSA7
0.0036 ± 0.0044,EState_VSA7
0.0034 ± 0.0025,PEOE_VSA7
0.0028 ± 0.0025,VSA_EState4
0.0027 ± 0.0067,PEOE_VSA8
0.0024 ± 0.0079,BCUT2D_MWLOW
0.0019 ± 0.0057,BCUT2D_MRLOW


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [18]:
feature_LGBM = ['PEOE_VSA6','MolLogP','MinPartialCharge','SMR_VSA7','EState_VSA7','PEOE_VSA7','VSA_EState4',
'PEOE_VSA8','BCUT2D_MWLOW','BCUT2D_MRLOW','fr_NH2','Ipc','NumSaturatedHeterocycles','SMR_VSA5','HallKierAlpha',
'BalabanJ','NumRotatableBonds','SlogP_VSA3','SMR_VSA10','EState_VSA8','EState_VSA2','BCUT2D_MWHI','MinAbsEStateIndex',
'EState_VSA10','EState_VSA6','fr_Ar_OH','FractionCSP3','BCUT2D_MRHI','NumHDonors','BCUT2D_CHGLO','EState_VSA5',
'MaxPartialCharge','MinEStateIndex','EState_VSA3','fr_bicyclic','NumAromaticHeterocycles','Chi0n','fr_Al_OH',
'SlogP_VSA2','PEOE_VSA3','NumSaturatedRings','fr_NH0','fr_sulfide','SlogP_VSA4','SlogP_VSA8','BCUT2D_LOGPHI',
'SMR_VSA1','VSA_EState8','PEOE_VSA1','EState_VSA9','FpDensityMorgan2','PEOE_VSA5','NumAliphaticRings',
'SlogP_VSA7','NumAromaticRings','SlogP_VSA11','PEOE_VSA13','fr_para_hydroxylation','fr_Ar_N','fr_NH1','fr_Ndealkylation2',
'fr_ester','fr_ether','PEOE_VSA12']

In [19]:
target_LGBM = df['CC50, mM']
data_LGBM = data[feature_LGBM]

In [20]:
x_train_LGBM, x_valid_LGBM = train_test_split(
    data_LGBM, train_size=0.8, random_state=1
)
y_train_LGBM, y_valid_LGBM = train_test_split(
    target_LGBM, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_LGBM.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_LGBM.shape))

x_train.shape = 798 rows, 64 cols
x_valid.shape = 200 rows, 64 cols


In [21]:
model_LGBM = LGBMClassifier(random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM)
# предсказание ответов для тренеровочной выборки
y_pred_LGBM_train = model_LGBM.predict(x_train_LGBM)
# предсказание ответов для тестовой выборки
y_pred_LGBM = model_LGBM.predict(x_valid_LGBM)

print("Train LGBM f1_score: ", f1_score(y_train_LGBM, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train_LGBM, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid_LGBM, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid_LGBM, y_pred_LGBM))

Train LGBM f1_score:  0.9702233250620347
Train LGBM accuracy_score:  0.9699248120300752
Validation LGBM f1_score:  0.7722772277227722
Validation LGBM accuracy_score:  0.77


Отобрав признаки мы улучшили метрики для модели LGBMClassifier на вадидации f1_score: 0.77, accuracy_score: 0.77.

**xgboost**

In [22]:
col = x_train.columns.tolist()
estimator_xgb = PermutationImportance(
    xgb_r, scoring= "neg_mean_squared_error", random_state=27
)
importance_xgb = estimator_xgb.fit(x_train, y_train)

In [23]:
show_weights(importance_xgb, feature_names=col, top=x_train.shape[1])

Weight,Feature
0.0170 ± 0.0052,BCUT2D_MWLOW
0.0065 ± 0.0058,PEOE_VSA6
0.0060 ± 0.0073,BCUT2D_MRLOW
0.0045 ± 0.0075,MolLogP
0.0040 ± 0.0019,fr_Al_OH
0.0040 ± 0.0040,PEOE_VSA7
0.0038 ± 0.0048,VSA_EState5
0.0030 ± 0.0026,PEOE_VSA3
0.0023 ± 0.0056,EState_VSA10
0.0020 ± 0.0058,EState_VSA2


Оставим только те признаки которые которые улучшают работу модели. И посмотрим как изменится метрика

In [24]:
feature_xgb = ['BCUT2D_MWLOW','PEOE_VSA6','BCUT2D_MRLOW','MolLogP','fr_Al_OH','PEOE_VSA7','VSA_EState5','PEOE_VSA3',
'EState_VSA10','EState_VSA2','SMR_VSA5','SMR_VSA6','VSA_EState1','NumRotatableBonds','NumSaturatedHeterocycles',
'VSA_EState4','EState_VSA9','MinAbsEStateIndex','EState_VSA6','VSA_EState7','NOCount','SlogP_VSA3','EState_VSA7',
'PEOE_VSA1','VSA_EState9','fr_bicyclic','NumAliphaticCarbocycles','SMR_VSA7','fr_allylic_oxid','fr_halogen',
'SMR_VSA10','AvgIpc','qed','SPS','fr_Ar_OH','PEOE_VSA2','EState_VSA8','fr_NH2','fr_NH1','EState_VSA5','fr_NH0',
'NumSaturatedRings','fr_amide','EState_VSA3','NumHDonors','SlogP_VSA2','SlogP_VSA11','HallKierAlpha','SlogP_VSA4',
'FractionCSP3','MinEStateIndex',]

In [25]:
target_xgb = df['CC50, mM']
data_xgb = data[feature_xgb]

In [26]:
x_train_xgb, x_valid_xgb = train_test_split(
    data_xgb, train_size=0.8, random_state=1
)
y_train_xgb, y_valid_xgb = train_test_split(
    target_xgb, train_size=0.8, random_state=1
)
print("x_train.shape = {} rows, {} cols".format(*x_train_xgb.shape))
print("x_valid.shape = {} rows, {} cols".format(*x_valid_xgb.shape))

x_train.shape = 798 rows, 51 cols
x_valid.shape = 200 rows, 51 cols


In [27]:
model_xgb = xg.XGBClassifier(random_state=27)
model_xgb.fit(x_train_xgb, y_train_xgb, verbose=100)
y_pred_xgb = model_xgb.predict(x_train_xgb)
y_val_xgb = model_xgb.predict(x_valid_xgb)
print("Train f1_score: ", f1_score(y_train_xgb, y_pred_xgb))
print("Train accuracy_score: ", accuracy_score(y_train_xgb, y_pred_xgb))
print("Validation f1_score: ", f1_score(y_valid_xgb, y_val_xgb))
print("Validation accuracy_score: ", accuracy_score(y_valid_xgb, y_val_xgb))

Train f1_score:  0.9702233250620347
Train accuracy_score:  0.9699248120300752
Validation f1_score:  0.7738693467336683
Validation accuracy_score:  0.775


Отобрав признаки мы улучшили метрики для модели xgboost на вадидации f1_score: 0.77, accuracy_score: 0.78.

# Подбор гиперпараметров

**CatBoost**

In [28]:
def catboost_cv(depth, learning_rate,n_estimators):
    depth = int(depth)
    n_estimators = int(n_estimators)
    model = CatBoostClassifier(
        depth=depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        eval_metric  = 'F1',
        random_seed=27
    )
    model.fit(x_train_cb, y_train_cb, verbose=100)

    return f1_score(y_valid_cb, model.predict(x_valid_cb))

In [29]:
param_space = {
    'depth': (2, 10),             
    'learning_rate': (0.001, 0.3),
    'n_estimators': (50, 2000)
}

bayesian_opt = BayesianOptimization(
    f=catboost_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=25)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   |   depth   | learni... | n_esti... |
-------------------------------------------------------------
0:	learn: 0.7087277	total: 18.1ms	remaining: 26.9s
100:	learn: 0.9702970	total: 612ms	remaining: 8.38s
200:	learn: 0.9702970	total: 1.17s	remaining: 7.45s
300:	learn: 0.9702970	total: 1.71s	remaining: 6.72s
400:	learn: 0.9701493	total: 2.23s	remaining: 6.03s
500:	learn: 0.9702970	total: 2.72s	remaining: 5.34s
600:	learn: 0.9702970	total: 3.28s	remaining: 4.82s
700:	learn: 0.9702970	total: 3.74s	remaining: 4.17s
800:	learn: 0.9702970	total: 4.2s	remaining: 3.58s
900:	learn: 0.9702970	total: 4.64s	remaining: 3.01s
1000:	learn: 0.9702970	total: 5.11s	remaining: 2.47s
1100:	learn: 0.9704433	total: 5.56s	remaining: 1.94s
1200:	learn: 0.9703704	total: 6.01s	remaining: 1.42s
1300:	learn: 0.9704433	total: 6.45s	remaining: 908ms
1400:	learn: 0.9701493	total: 6.89s	remaining: 408ms
1483:	learn: 0.9702233	total: 7.25s	remaining: 0us
| 1         | 0.7745    | 5.406     | 0.24

1000:	learn: 0.9678218	total: 1.61s	remaining: 1.29s
1100:	learn: 0.9678218	total: 1.84s	remaining: 1.17s
1200:	learn: 0.9702970	total: 2s	remaining: 1.01s
1300:	learn: 0.9702970	total: 2.18s	remaining: 842ms
1400:	learn: 0.9702970	total: 2.32s	remaining: 667ms
1500:	learn: 0.9702970	total: 2.47s	remaining: 498ms
1600:	learn: 0.9702970	total: 2.61s	remaining: 331ms
1700:	learn: 0.9702970	total: 2.76s	remaining: 167ms
1800:	learn: 0.9702970	total: 3.01s	remaining: 5.02ms
1803:	learn: 0.9702970	total: 3.02s	remaining: 0us
| 11        | 0.76      | 2.028     | 0.07283   | 1.804e+03 |
0:	learn: 0.7156489	total: 2.41ms	remaining: 120ms
50:	learn: 0.8627936	total: 81.9ms	remaining: 0us
| 12        | 0.7411    | 2.51      | 0.2802    | 51.34     |
0:	learn: 0.7156489	total: 1.8ms	remaining: 1.16s
100:	learn: 0.9117647	total: 151ms	remaining: 814ms
200:	learn: 0.9557740	total: 303ms	remaining: 669ms
300:	learn: 0.9690977	total: 473ms	remaining: 541ms
400:	learn: 0.9702970	total: 618ms	remainin

500:	learn: 0.9702233	total: 6.8s	remaining: 1.67s
600:	learn: 0.9704433	total: 8.05s	remaining: 308ms
623:	learn: 0.9702970	total: 8.37s	remaining: 0us
| 22        | 0.7562    | 7.137     | 0.2241    | 624.8     |
0:	learn: 0.7087277	total: 7.13ms	remaining: 969ms
100:	learn: 0.9702970	total: 508ms	remaining: 181ms
136:	learn: 0.9704433	total: 706ms	remaining: 0us
| 23        | 0.7451    | 5.513     | 0.2198    | 137.7     |
0:	learn: 0.7005291	total: 4.5ms	remaining: 1.23s
100:	learn: 0.9702970	total: 302ms	remaining: 518ms
200:	learn: 0.9702233	total: 597ms	remaining: 217ms
273:	learn: 0.9702233	total: 838ms	remaining: 0us
| 24        | 0.7822    | 4.104     | 0.2313    | 274.6     |
0:	learn: 0.7206385	total: 35.2ms	remaining: 36.3s
100:	learn: 0.9374233	total: 2.67s	remaining: 24.6s
200:	learn: 0.9640644	total: 5.3s	remaining: 21.9s
300:	learn: 0.9702970	total: 7.95s	remaining: 19.3s
400:	learn: 0.9702970	total: 10.6s	remaining: 16.6s
500:	learn: 0.9702970	total: 13.2s	remaining: 

In [30]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'l2_leaf_reg'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'depth': 7, 'learning_rate': 0.2661536369860892, 'n_estimators': 1723.124788664315}
Best R-squared Score: 0.7864


In [31]:
params = {
    'depth': 7,             
    'learning_rate': 0.2661536369860892, 
    'n_estimators': 1723
}
model = CatBoostClassifier(**params, random_state=27)
model.fit(x_train_cb, y_train_cb, verbose=100)
y_pred_cb = model.predict(x_train_cb)
y_val_cb = model.predict(x_valid_cb)
print("Train сat_Boost f1_score: ", f1_score(y_train_cb, y_pred_cb))
print("Train сat_Boost accuracy_score: ", accuracy_score(y_train_cb, y_pred_cb))
print("Validation сat_Boost f1_score: ", f1_score(y_valid_cb, y_val_cb))
print("Validation сat_Boost accuracy_score: ", accuracy_score(y_valid_cb, y_val_cb))

0:	learn: 0.6282430	total: 19.6ms	remaining: 33.7s
100:	learn: 0.0807359	total: 1.43s	remaining: 23s
200:	learn: 0.0626392	total: 2.76s	remaining: 20.9s
300:	learn: 0.0589864	total: 4.11s	remaining: 19.4s
400:	learn: 0.0573007	total: 5.51s	remaining: 18.2s
500:	learn: 0.0567344	total: 6.92s	remaining: 16.9s
600:	learn: 0.0563437	total: 8.28s	remaining: 15.5s
700:	learn: 0.0560691	total: 9.62s	remaining: 14s
800:	learn: 0.0559058	total: 11s	remaining: 12.6s
900:	learn: 0.0557205	total: 12.3s	remaining: 11.2s
1000:	learn: 0.0555831	total: 13.6s	remaining: 9.84s
1100:	learn: 0.0554654	total: 14.9s	remaining: 8.44s
1200:	learn: 0.0554134	total: 16.3s	remaining: 7.08s
1300:	learn: 0.0553675	total: 17.6s	remaining: 5.72s
1400:	learn: 0.0553318	total: 18.9s	remaining: 4.35s
1500:	learn: 0.0553071	total: 20.5s	remaining: 3.03s
1600:	learn: 0.0552628	total: 21.8s	remaining: 1.66s
1700:	learn: 0.0552347	total: 23.3s	remaining: 301ms
1722:	learn: 0.0552241	total: 23.6s	remaining: 0us
Train сat_Bo

После подбора гиперпараметров мы незначительно улучшили метрики модели CatBoost на вадидации f1_score: 0.786, accuracy_score: 0.78.



**LGBM**

In [32]:
def LGBM_cv(max_depth, learning_rate, n_estimators):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = LGBMClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,          
        eval_metric  = 'F1',
        random_seed=27
    )
    model.fit(x_train_LGBM, y_train_LGBM, eval_set = (x_valid_LGBM, y_valid_LGBM))
    return f1_score(y_valid_LGBM, model.predict(x_valid_LGBM))

In [33]:
param_space = {
    'max_depth': (2, 10),            
    'learning_rate': (0.001, 0.3),
    'n_estimators': (50, 2000)
}

bayesian_opt = BayesianOptimization(
    f=LGBM_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=50)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------
[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's binary_logloss: 0.656695
[2]	valid_0's binary_logloss: 0.623143
[3]	valid_0's binary_logloss: 0.596106
[4]	valid_0's binary_logloss: 0.571459
[5]	valid_0's binary_logloss: 0.557189
[6]	valid_0's binary_logloss: 0.543868
[7]	valid_0's binary_logloss: 0.526205
[8]	valid_0's binary_logloss: 0.51654
[9]	valid_0's binary_logloss: 0.510353
[10]	valid_0's binary_logloss: 0.497961
[11]	valid_0's binary_logloss: 0.491303
[12]	valid_0's binary_logloss: 0.488841
[13]	valid_0's binary_logloss: 0.48929
[14]	valid_0's binary_logloss: 0.488627
[15]	valid_0's binary_logloss: 0.487116
[16]	valid_0's binary_logloss: 0.486606
[17]	valid_0's binary_logloss: 0.488827
[18]	valid_0's binary_logloss: 0.485225
[19]	valid_0's binary_logloss: 0.482083
[20]	valid_0's binary_logloss: 0.476905
[21]	valid_0's binary_logloss: 0.47

[216]	valid_0's binary_logloss: 0.807417
[217]	valid_0's binary_logloss: 0.809377
[218]	valid_0's binary_logloss: 0.8106
[219]	valid_0's binary_logloss: 0.814086
[220]	valid_0's binary_logloss: 0.814744
[221]	valid_0's binary_logloss: 0.819844
[222]	valid_0's binary_logloss: 0.820764
[223]	valid_0's binary_logloss: 0.822251
[224]	valid_0's binary_logloss: 0.822189
[225]	valid_0's binary_logloss: 0.823979
[226]	valid_0's binary_logloss: 0.823354
[227]	valid_0's binary_logloss: 0.82325
[228]	valid_0's binary_logloss: 0.822619
[229]	valid_0's binary_logloss: 0.827286
[230]	valid_0's binary_logloss: 0.828681
[231]	valid_0's binary_logloss: 0.830261
[232]	valid_0's binary_logloss: 0.83203
[233]	valid_0's binary_logloss: 0.83266
[234]	valid_0's binary_logloss: 0.833597
[235]	valid_0's binary_logloss: 0.83394
[236]	valid_0's binary_logloss: 0.834721
[237]	valid_0's binary_logloss: 0.839435
[238]	valid_0's binary_logloss: 0.843106
[239]	valid_0's binary_logloss: 0.844327
[240]	valid_0's binary

[528]	valid_0's binary_logloss: 1.1411
[529]	valid_0's binary_logloss: 1.14158
[530]	valid_0's binary_logloss: 1.14413
[531]	valid_0's binary_logloss: 1.14499
[532]	valid_0's binary_logloss: 1.14537
[533]	valid_0's binary_logloss: 1.14564
[534]	valid_0's binary_logloss: 1.14556
[535]	valid_0's binary_logloss: 1.14663
[536]	valid_0's binary_logloss: 1.14691
[537]	valid_0's binary_logloss: 1.14915
[538]	valid_0's binary_logloss: 1.15135
[539]	valid_0's binary_logloss: 1.15244
[540]	valid_0's binary_logloss: 1.1525
[541]	valid_0's binary_logloss: 1.15276
[542]	valid_0's binary_logloss: 1.15283
[543]	valid_0's binary_logloss: 1.1531
[544]	valid_0's binary_logloss: 1.15298
[545]	valid_0's binary_logloss: 1.15322
[546]	valid_0's binary_logloss: 1.15329
[547]	valid_0's binary_logloss: 1.15477
[548]	valid_0's binary_logloss: 1.15468
[549]	valid_0's binary_logloss: 1.15471
[550]	valid_0's binary_logloss: 1.15441
[551]	valid_0's binary_logloss: 1.15447
[552]	valid_0's binary_logloss: 1.15554
[55

[824]	valid_0's binary_logloss: 1.20888
[825]	valid_0's binary_logloss: 1.20891
[826]	valid_0's binary_logloss: 1.20895
[827]	valid_0's binary_logloss: 1.20892
[828]	valid_0's binary_logloss: 1.20893
[829]	valid_0's binary_logloss: 1.20893
[830]	valid_0's binary_logloss: 1.21033
[831]	valid_0's binary_logloss: 1.21032
[832]	valid_0's binary_logloss: 1.21024
[833]	valid_0's binary_logloss: 1.21028
[834]	valid_0's binary_logloss: 1.21027
[835]	valid_0's binary_logloss: 1.21029
[836]	valid_0's binary_logloss: 1.21032
[837]	valid_0's binary_logloss: 1.21033
[838]	valid_0's binary_logloss: 1.2104
[839]	valid_0's binary_logloss: 1.21041
[840]	valid_0's binary_logloss: 1.21057
[841]	valid_0's binary_logloss: 1.20986
[842]	valid_0's binary_logloss: 1.20985
[843]	valid_0's binary_logloss: 1.20993
[844]	valid_0's binary_logloss: 1.20996
[845]	valid_0's binary_logloss: 1.21051
[846]	valid_0's binary_logloss: 1.21066
[847]	valid_0's binary_logloss: 1.21068
[848]	valid_0's binary_logloss: 1.21072
[

[1169]	valid_0's binary_logloss: 1.23358
[1170]	valid_0's binary_logloss: 1.2336
[1171]	valid_0's binary_logloss: 1.23359
[1172]	valid_0's binary_logloss: 1.23362
[1173]	valid_0's binary_logloss: 1.23369
[1174]	valid_0's binary_logloss: 1.23371
[1175]	valid_0's binary_logloss: 1.2337
[1176]	valid_0's binary_logloss: 1.23374
[1177]	valid_0's binary_logloss: 1.23376
[1178]	valid_0's binary_logloss: 1.23377
[1179]	valid_0's binary_logloss: 1.2338
[1180]	valid_0's binary_logloss: 1.23387
[1181]	valid_0's binary_logloss: 1.23392
[1182]	valid_0's binary_logloss: 1.23406
[1183]	valid_0's binary_logloss: 1.23405
[1184]	valid_0's binary_logloss: 1.23402
[1185]	valid_0's binary_logloss: 1.23405
[1186]	valid_0's binary_logloss: 1.23412
[1187]	valid_0's binary_logloss: 1.2339
[1188]	valid_0's binary_logloss: 1.23392
[1189]	valid_0's binary_logloss: 1.23398
[1190]	valid_0's binary_logloss: 1.23403
[1191]	valid_0's binary_logloss: 1.23699
[1192]	valid_0's binary_logloss: 1.23701
[1193]	valid_0's bin

...

[326]	valid_0's binary_logloss: 0.568562
[327]	valid_0's binary_logloss: 0.567909
[328]	valid_0's binary_logloss: 0.56884
[329]	valid_0's binary_logloss: 0.56814
[330]	valid_0's binary_logloss: 0.569287
[331]	valid_0's binary_logloss: 0.570067
[332]	valid_0's binary_logloss: 0.570063
[333]	valid_0's binary_logloss: 0.569891
[334]	valid_0's binary_logloss: 0.57052
[335]	valid_0's binary_logloss: 0.570705
[336]	valid_0's binary_logloss: 0.571006
[337]	valid_0's binary_logloss: 0.569915
[338]	valid_0's binary_logloss: 0.570111
[339]	valid_0's binary_logloss: 0.570892
[340]	valid_0's binary_logloss: 0.570479
[341]	valid_0's binary_logloss: 0.571675
[342]	valid_0's binary_logloss: 0.571522
[343]	valid_0's binary_logloss: 0.572107
[344]	valid_0's binary_logloss: 0.572432
[345]	valid_0's binary_logloss: 0.573354
[346]	valid_0's binary_logloss: 0.574262
[347]	valid_0's binary_logloss: 0.575312
[348]	valid_0's binary_logloss: 0.574864
[349]	valid_0's binary_logloss: 0.574738
[350]	valid_0's bin

[708]	valid_0's binary_logloss: 0.739525
[709]	valid_0's binary_logloss: 0.739092
[710]	valid_0's binary_logloss: 0.738918
[711]	valid_0's binary_logloss: 0.739266
[712]	valid_0's binary_logloss: 0.740389
[713]	valid_0's binary_logloss: 0.741041
[714]	valid_0's binary_logloss: 0.74292
[715]	valid_0's binary_logloss: 0.744149
[716]	valid_0's binary_logloss: 0.744267
[717]	valid_0's binary_logloss: 0.746006
[718]	valid_0's binary_logloss: 0.746093
[719]	valid_0's binary_logloss: 0.746435
[720]	valid_0's binary_logloss: 0.745614
[721]	valid_0's binary_logloss: 0.746569
[722]	valid_0's binary_logloss: 0.747239
[723]	valid_0's binary_logloss: 0.74619
[724]	valid_0's binary_logloss: 0.746307
[725]	valid_0's binary_logloss: 0.746418
[726]	valid_0's binary_logloss: 0.748077
[727]	valid_0's binary_logloss: 0.749315
[728]	valid_0's binary_logloss: 0.749035
[729]	valid_0's binary_logloss: 0.751005
[730]	valid_0's binary_logloss: 0.751161
[731]	valid_0's binary_logloss: 0.752263
[732]	valid_0's bi

[1260]	valid_0's binary_logloss: 0.95962
[1261]	valid_0's binary_logloss: 0.960157
[1262]	valid_0's binary_logloss: 0.960068
[1263]	valid_0's binary_logloss: 0.960098
[1264]	valid_0's binary_logloss: 0.960227
[1265]	valid_0's binary_logloss: 0.960983
[1266]	valid_0's binary_logloss: 0.961361
[1267]	valid_0's binary_logloss: 0.962771
[1268]	valid_0's binary_logloss: 0.962208
[1269]	valid_0's binary_logloss: 0.962364
[1270]	valid_0's binary_logloss: 0.962938
[1271]	valid_0's binary_logloss: 0.962843
[1272]	valid_0's binary_logloss: 0.962616
[1273]	valid_0's binary_logloss: 0.962437
[1274]	valid_0's binary_logloss: 0.962498
[1275]	valid_0's binary_logloss: 0.962695
[1276]	valid_0's binary_logloss: 0.962721
[1277]	valid_0's binary_logloss: 0.962963
[1278]	valid_0's binary_logloss: 0.962876
[1279]	valid_0's binary_logloss: 0.962873
[1280]	valid_0's binary_logloss: 0.962943
[1281]	valid_0's binary_logloss: 0.962949
[1282]	valid_0's binary_logloss: 0.962947
[1283]	valid_0's binary_logloss: 0.

[LightGBM] [Warning] Unknown parameter: eval_metric
[1]	valid_0's binary_logloss: 0.69169
[2]	valid_0's binary_logloss: 0.690165
[3]	valid_0's binary_logloss: 0.688685
[4]	valid_0's binary_logloss: 0.687248
[5]	valid_0's binary_logloss: 0.685852
[6]	valid_0's binary_logloss: 0.684497
[7]	valid_0's binary_logloss: 0.683248
[8]	valid_0's binary_logloss: 0.681957
[9]	valid_0's binary_logloss: 0.680776
[10]	valid_0's binary_logloss: 0.67941
[11]	valid_0's binary_logloss: 0.678294
[12]	valid_0's binary_logloss: 0.676752
[13]	valid_0's binary_logloss: 0.675249
[14]	valid_0's binary_logloss: 0.6742
[15]	valid_0's binary_logloss: 0.67275
[16]	valid_0's binary_logloss: 0.671338
[17]	valid_0's binary_logloss: 0.669961
[18]	valid_0's binary_logloss: 0.668994
[19]	valid_0's binary_logloss: 0.667665
[20]	valid_0's binary_logloss: 0.666369
[21]	valid_0's binary_logloss: 0.665133
[22]	valid_0's binary_logloss: 0.664234
[23]	valid_0's binary_logloss: 0.663013
[24]	valid_0's binary_logloss: 0.661848
[2

[394]	valid_0's binary_logloss: 0.529437
[395]	valid_0's binary_logloss: 0.52929
[396]	valid_0's binary_logloss: 0.529018
[397]	valid_0's binary_logloss: 0.529171
[398]	valid_0's binary_logloss: 0.529125
[399]	valid_0's binary_logloss: 0.528821
[400]	valid_0's binary_logloss: 0.528613
[401]	valid_0's binary_logloss: 0.528493
[402]	valid_0's binary_logloss: 0.528264
[403]	valid_0's binary_logloss: 0.528127
[404]	valid_0's binary_logloss: 0.527864
[405]	valid_0's binary_logloss: 0.527368
[406]	valid_0's binary_logloss: 0.527523
[407]	valid_0's binary_logloss: 0.527463
[408]	valid_0's binary_logloss: 0.5274
[409]	valid_0's binary_logloss: 0.527285
[410]	valid_0's binary_logloss: 0.527102
[411]	valid_0's binary_logloss: 0.526879
[412]	valid_0's binary_logloss: 0.526623
[413]	valid_0's binary_logloss: 0.52678
[414]	valid_0's binary_logloss: 0.526704
[415]	valid_0's binary_logloss: 0.526503
[416]	valid_0's binary_logloss: 0.526393
[417]	valid_0's binary_logloss: 0.526266
[418]	valid_0's bina

In [34]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'n_estimators'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.1213437386005643, 'max_depth': 7.990207206075692, 'n_estimators': 170}
Best R-squared Score: 0.7783


In [35]:
params = {
    'max_depth': 7,            
    'learning_rate': 0.1213437386005643,  
    'n_estimators': 170
}

model_LGBM = LGBMClassifier(**params, random_state=27)
model_LGBM.fit(x_train_LGBM, y_train_LGBM)
# предсказание ответов для тренеровочной выборки
y_pred_LGBM_train = model_LGBM.predict(x_train_LGBM)
# предсказание ответов для тестовой выборки
y_pred_LGBM = model_LGBM.predict(x_valid_LGBM)

print("Train LGBM f1_score: ", f1_score(y_train_LGBM, y_pred_LGBM_train))
print("Train LGBM accuracy_score: ", accuracy_score(y_train_LGBM, y_pred_LGBM_train))
print("Validation LGBM f1_score: ", f1_score(y_valid_LGBM, y_pred_LGBM))
print("Validation LGBM accuracy_score: ", accuracy_score(y_valid_LGBM, y_pred_LGBM))

Train LGBM f1_score:  0.9703703703703703
Train LGBM accuracy_score:  0.9699248120300752
Validation LGBM f1_score:  0.7783251231527093
Validation LGBM accuracy_score:  0.775


После подбора гиперпараметров мы незначительно улучшили метрики LGBMClassifier на вадидации f1_score: 0.778, accuracy_score: 0.775.



**xgboost**

In [36]:
def XGB_cv(max_depth, learning_rate, n_estimators):
    max_depth = int(max_depth)
    n_estimators = int(n_estimators)
    model = xg.XGBClassifier(
        max_depth=max_depth,
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        random_seed=27
    )
    model.fit(x_train_xgb, y_train_xgb)
    return f1_score(y_valid_xgb, model.predict(x_valid_xgb))

In [37]:
param_space = {
    'max_depth': (2, 10),            
    'learning_rate': (0.001, 0.3),  
    'n_estimators': (10, 2000)
}

bayesian_opt = BayesianOptimization(
    allow_duplicate_points=True,f=XGB_cv, pbounds=param_space, random_state=27)
bayesian_opt.maximize(init_points=5, n_iter=50)
results = pd.DataFrame(bayesian_opt.res)
results.sort_values(by='target', ascending=False, inplace=True)

|   iter    |  target   | learni... | max_depth | n_esti... |
-------------------------------------------------------------


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [21:37:12] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 1         | 0.7624    | 0.1283    | 8.517     | 1.473e+03 |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [21:37:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 2         | 0.7685    | 0.2605    | 5.067     | 1.959e+03 |


...

| 53        | 0.731     | 0.0836    | 6.92      | 136.5     |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [21:38:39] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 54        | 0.7562    | 0.2578    | 4.306     | 1.959e+03 |


/home/rik/.local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [21:38:41] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "random_seed" } are not used.

  warnings.warn(smsg, UserWarning)


| 55        | 0.7488    | 0.01312   | 2.343     | 697.7     |


In [38]:
best_hyperparameters = bayesian_opt.max
best_hyperparameters['params'] = {param: int(value) if param in [
    'depth', 'iterations', 'n_estimators'] else value for param, value in best_hyperparameters['params'].items()}
print("Best hyperparameters:", best_hyperparameters['params'])
print(f"Best R-squared Score: {best_hyperparameters['target']:.4f}")

Best hyperparameters: {'learning_rate': 0.24156191458213627, 'max_depth': 8.44220920786574, 'n_estimators': 1014}
Best R-squared Score: 0.7800


In [39]:
params = {
    'max_depth': 8,            
    'learning_rate':0.24156191458213627
    ,  
    'n_estimators': 1014  
}
model_XGB = xg.XGBClassifier(**params, random_state=27)
model_XGB.fit(x_train_xgb, y_train_xgb)
y_pred_xgb = model_XGB.predict(x_train_xgb)
y_val_xgb = model_XGB.predict(x_valid_xgb)
print("Train f1_score: ", f1_score(y_train_xgb, y_pred_xgb))
print("Train accuracy_score: ", accuracy_score(y_train_xgb, y_pred_xgb))
print("Validation f1_score: ", f1_score(y_valid_xgb, y_val_xgb))
print("Validation accuracy_score: ", accuracy_score(y_valid_xgb, y_val_xgb))

Train f1_score:  0.9703703703703703
Train accuracy_score:  0.9699248120300752
Validation f1_score:  0.7799999999999999
Validation accuracy_score:  0.78


После подбора гиперпараметров мы улучшили метрики для модели XGBClassifier на вадидации f1_score: 0.78, accuracy_score: 0.78.

**Выводы**

Мы сравнили 4 регрессионые модели: DecisionTreeRegressor, CatBoostRegressor, LGBMRegressor, XGBRegressor. Наилучший результат показала модель CatBoostClassifier с результатом на трейне: f1_score= 0.97 и accuracy_score= 0.97 и тесте: f1_score=0.7864, accuracy_score=0.78.

**Рекомендации**

Для улучшения значения метрики и снижения переобучения нужно будет увеличить количество данных.